In [1]:
import numpy as np
import cv2
import pandas as pd


def feature_extraction(img):
    df = pd.DataFrame()
    # Add original pixel values to the data frame as feature 1
    img2 = img.reshape(-1)
    df['Original_Image'] = img2
    
    # Add other features

    # Gabor features
    num = 1
    kernels = []
    for theta in range(2):
        theta = theta/4. * np.pi
        for sigma in (1, 3):
            for lamda in np.arange(0, np.pi, np.pi/4):
                for gamma in (0.05, 0.5):
                    #print(theta, sigma, lamda, gamma)
                    gabor_label = 'Gabor' + str(num)
                    kernel = cv2.getGaborKernel((9, 9), sigma, theta, lamda, gamma, 0, ktype=cv2.CV_32F)
                    kernels.append(kernel)

                    fimg = cv2.filter2D(img, cv2.CV_8UC3, kernel)
                    filtered_img = fimg.reshape(-1)
                    df[gabor_label] = filtered_img
                    num += 1
                    # print(gabor_label)

    # Canny edge
    edges = cv2.Canny(img, 100, 200)
    edges1 = edges.reshape(-1)
    df['Canny Edge'] = edges1

    from skimage.filters import roberts, sobel, scharr, prewitt

    edge_roberts = roberts(img)
    edge_roberts1 = edge_roberts.reshape(-1)
    df['Roberts'] = edge_roberts1

    edge_sobel = sobel(img)
    edge_sobel1 = edge_sobel.reshape(-1)
    df['sobel'] = edge_sobel1

    edge_scharr = scharr(img)
    edge_scharr1 = edge_scharr.reshape(-1)
    df['scharr'] = edge_scharr1

    edge_prewitt = prewitt(img)
    edge_prewitt1 = edge_prewitt.reshape(-1)
    df['prewitt'] = edge_prewitt1
    
    from scipy import ndimage as nd

    gaussian_img = nd.gaussian_filter(img, sigma=3)
    gaussian_img1 = gaussian_img.reshape(-1)
    df['gaussian_s3'] = gaussian_img1

    gaussian_img2 = nd.gaussian_filter(img, sigma=7)
    gaussian_img3 = gaussian_img2.reshape(-1)
    df['gaussian_s7'] = gaussian_img3

    # median filter
    median_img = nd.median_filter(img, size=3)
    median_img2 = median_img.reshape(-1)
    df['median_img'] = median_img2
    
    return df
    

In [2]:
import glob
import pickle
import matplotlib.pyplot as plt

filename = 'sandstone_model'
load_model = pickle.load(open(filename, 'rb'))

path = 'image/Train_images/*.tif'

#num = 1

for file in glob.glob(path):
    img1 = cv2.imread(file)
    img = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    
    X = feature_extraction(img)
    result = load_model.predict(X)
    segmented = result.reshape((img.shape))
    
    name = file.split("e_")
    name = name[1].split(".")
    plt.imsave('image/Segmented/' + name[0] + ".jpg",
               segmented,
               cmap='jet')
    #num += 1



